In [ ]:
pip install langchain langchain_community langchain-openai langchain_chroma langchainhub

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 973.5/973.5 kB 7.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 45.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 310.2/310.2 kB 19.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 124.4/124.4 kB 6.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 320.7/320.7 kB 18.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 37.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 526.8/526.8 kB 24.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 92.0/92.0 kB 5.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 26.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.4/62.4 kB 3.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.3/41.3 kB 2.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.8/6.8 MB 17.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━

### LangSmith

使用LangChain构建的许多应用程序将都包含多个步骤和多次调用LLM调用。随着这些应用程序变得越来越复杂，能够检查chain或agent内部发生的细节变得至关重要。这样做的最佳方式是使用[LangSmith](https://smith.langchain.com/)。

请注意，LangSmith并非必需，它只是在我们开发调试应用的时候非常有用。如果想使用可以在[官网](https://smith.langchain.com/)注册后申请秘钥，每个月都会有一定的免费使用额度，足够我们学习和测试，将key设置在的环境变量中就可以轻松使用LangSmith。

notebook中设置

In [ ]:
import os
from google.colab import userdata
os.environ["OPENAI_API_KEY"] = userdata.get('OPENAI_API_KEY')
os.environ["OPENAI_API_BASE"] = userdata.get('OPENAI_API_BASE')
os.environ["LANGCHAIN_TRACING_V2"] = "true"
os.environ["LANGCHAIN_API_KEY"] = userdata.get('LANGCHAIN_API_KEY')

## Preview

在本指南中，我们将基于网页上构建一个问答应用程序。

我们可以创建一个简单的本地知识库索引和 RAG 链来实现这个功能，这大约只需要 20 行代码。

- OpenAI

In [ ]:
from langchain_openai import ChatOpenAI
llm = ChatOpenAI(model="gpt-3.5-turbo-0125")

In [ ]:
llm.invoke("什么是Tree of Thoughts?")

AIMessage(content='Tree of Thoughts是一种概念，代表一个人的思维模式和思想体系。这个概念类似于一个树，树干代表一个人的核心信念和价值观，树枝代表不同的思维路径和分支，叶子代表具体的想法和观点。通过理解和研究一个人的Tree of Thoughts，可以更好地了解他们的思维方式和决策过程。这个概念也可以帮助人们更好地管理自己的思维，并促进思维的发展和成长。', response_metadata={'token_usage': {'completion_tokens': 171, 'prompt_tokens': 15, 'total_tokens': 186}, 'model_name': 'gpt-3.5-turbo-0125', 'system_fingerprint': 'fp_811936bd4f', 'finish_reason': 'stop', 'logprobs': None}, id='run-d2a1b112-10d3-4336-8002-7633dd610b75-0', usage_metadata={'input_tokens': 15, 'output_tokens': 171, 'total_tokens': 186})

In [ ]:
import bs4
from langchain_chroma import Chroma
from langchain_community.document_loaders import WebBaseLoader
from langchain_openai import OpenAIEmbeddings
from langchain_text_splitters import RecursiveCharacterTextSplitter
# Load, chunk and index the contents of the blog.
loader = WebBaseLoader(
    web_paths=("https://lilianweng.github.io/posts/2023-06-23-agent/",),
    bs_kwargs=dict(
        parse_only=bs4.SoupStrainer(
            class_=("post-content", "post-title", "post-header")
)
),
)
docs = loader.load()
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
splits = text_splitter.split_documents(docs)
vectorstore = Chroma.from_documents(documents=splits, embedding=OpenAIEmbeddings())
# Retrieve and generate using the relevant snippets of the blog.
retriever = vectorstore.as_retriever()


In [ ]:
from langchain import hub
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough
prompt = hub.pull("rlm/rag-prompt")

def format_docs(docs):
  return "\n\n".join(doc.page_content for doc in docs)

rag_chain = (
{"context": retriever | format_docs, "question": RunnablePassthrough()}
| prompt
| llm
| StrOutputParser()
)
rag_chain.invoke("什么是Tree of Thoughts?")

'Tree of Thoughts 是一种扩展了CoT的技术，通过在每个步骤探索多种推理可能性来拆分问题，生成多个思考的树结构。搜索过程可以采用BFS或DFS，每个状态由分类器或多数投票评估。ReAct在知识密集型任务和决策任务上的实验中表现更好，其中Thought步骤被移除。Reflexion是一个框架，为智能体提供动态记忆和自我反思能力，以提高推理能力。'

**API 调用:**[WebBaseLoader](https://api.python.langchain.com/en/latest/document_loaders/langchain_community.document_loaders.web_base.WebBaseLoader.html) | [StrOutputParser](https://api.python.langchain.com/en/latest/output_parsers/langchain_core.output_parsers.string.StrOutputParser.html) | [RunnablePassthrough](https://api.python.langchain.com/en/latest/runnables/langchain_core.runnables.passthrough.RunnablePassthrough.html) | [OpenAIEmbeddings](https://api.python.langchain.com/en/latest/embeddings/langchain_openai.embeddings.base.OpenAIEmbeddings.html) | [RecursiveCharacterTextSplitter](https://api.python.langchain.com/en/latest/character/langchain_text_splitters.character.RecursiveCharacterTextSplitter.html)

In [ ]:
# cleanup
vectorstore.delete_collection()

查看 [LangSmith trace](https://smith.langchain.com/public/1c6ca97e-445b-4d00-84b4-c7befcbc59fe/r).

## 详细流程

让我们逐步解释上面的代码，以便真正理解代码到底进行了什么操作。

## 1. 创建索引: 加载数据

加载数据我们可以使用文档加载器（[DocumentLoaders](https://python.langchain.com/v0.2/docs/concepts/#document-loaders)）来完成，它们是从源加载数据并返回[Documents](https://api.python.langchain.com/en/latest/documents/langchain_core.documents.base.Document.html)列表的对象。`Document` 是一个带有 `page_content` (str) 和 `metadata` (dict) 的对象。

对于加载网页内容，我们将使用[WebBaseLoader](https://python.langchain.com/v0.2/docs/integrations/document_loaders/web_base/)（属于文档加载器的一种），它使用 `urllib` 从 web URL 加载 HTML，然后使用 `BeautifulSoup` 将其解析为文本。我们可以通过向 `bs_kwargs` 传递参数来自定义 HTML -> 文本解析（参见[BeautifulSoup文档](https://beautiful-soup-4.readthedocs.io/en/latest/#beautifulsoup)）。在这种情况下，只有具有“post-content”、“post-title”或“post-header”类的 HTML 标签是相关的，所以我们将删除所有其他标签。

In [ ]:
import bs4
from langchain_community.document_loaders import WebBaseLoader
# Only keep post title, headers, and content from the full HTML.
bs4_strainer = bs4.SoupStrainer(class_=("post-title", "post-header", "post-content"))
loader = WebBaseLoader(
    web_paths=("https://lilianweng.github.io/posts/2023-06-23-agent/",),
    bs_kwargs={"parse_only": bs4_strainer},
)
docs = loader.load()
len(docs[0].page_content)


43131

**API 调用:**[WebBaseLoader](https://api.python.langchain.com/en/latest/document_loaders/langchain_community.document_loaders.web_base.WebBaseLoader.html)

In [ ]:
print(docs[0].page_content[:500])



      LLM Powered Autonomous Agents
    
Date: June 23, 2023  |  Estimated Reading Time: 31 min  |  Author: Lilian Weng


Building agents with LLM (large language model) as its core controller is a cool concept. Several proof-of-concepts demos, such as AutoGPT, GPT-Engineer and BabyAGI, serve as inspiring examples. The potentiality of LLM extends beyond generating well-written copies, stories, essays and programs; it can be framed as a powerful general problem solver.
Agent System Overview#
In


### `DocumentLoader`

`DocumentLoader`: 将数据源加载成`Documents` 文本列表

- [Docs](https://python.langchain.com/v0.2/docs/how_to/#document-loaders): 怎样使用文档加载器 `DocumentLoaders`.
- [Integrations](https://python.langchain.com/v0.2/docs/integrations/document_loaders/): 160+ 可供选择的文档类型集成。
- [Interface](https://api.python.langchain.com/en/latest/document_loaders/langchain_core.document_loaders.base.BaseLoader.html): API 的基本接口。

## 2. 创建索引: 分割

我们的加载文档超过42,000个字符。这太长了，很多模型的上下文窗口无法完全容纳。即使是那些可以完全容纳整篇文章的模型，也很难在非常长的输入中找到信息。

为了处理这个问题，我们将把“文档”分成多个块进行嵌入和向量存储。这样可以帮助我们在运行时只检索出最相关的部分博客文章。

在这种情况下，我们将把我们的文档分成每1000个字符一组，每组之间有200个字符的重叠。重叠有助于减少分离语句与与之相关的重要上下文的可能性。我们使用了[RecursiveCharacterTextSplitter](https://python.langchain.com/v0.2/docs/how_to/recursive_text_splitter/)，它将递归地使用常见的分隔符（例如换行符）拆分文档，直到每个块的大小合适。这是通用文本用例的推荐文本拆分器。

我们设置 `add_start_index=True`，这样每个拆分的文档在初始文档中开始的字符索引位置将被保留为元数据 `start_index`属性。

In [ ]:
from langchain_text_splitters import RecursiveCharacterTextSplitter
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=1000, chunk_overlap=200, add_start_index=True
)
all_splits = text_splitter.split_documents(docs)
len(all_splits)

66

**API Reference:**[RecursiveCharacterTextSplitter](https://api.python.langchain.com/en/latest/character/langchain_text_splitters.character.RecursiveCharacterTextSplitter.html)

In [ ]:
len(all_splits[0].page_content)

969

In [ ]:
all_splits[10].metadata

{'source': 'https://lilianweng.github.io/posts/2023-06-23-agent/',
 'start_index': 7056}

### `TextSplitter`

`TextSplitter` （ `DocumentTransformer`的子类）: 将`Document`列表分割为更小的块的对象。

- 探索[上下文感知分割器](https://python.langchain.com/v0.2/docs/how_to/#text-splitters)，它们保留原始文档中每个分割的位置（“context”）。
- [Code (py or js)](https://python.langchain.com/v0.2/docs/integrations/document_loaders/source_code/)
- [Scientific papers](https://python.langchain.com/v0.2/docs/integrations/document_loaders/grobid/)
- [Interface](https://api.python.langchain.com/en/latest/base/langchain_text_splitters.base.TextSplitter.html): API 的基本接口。

`DocumentTransformer`: 执行对不同类型文档进行切割转换的对象。

- [Docs](https://python.langchain.com/v0.2/docs/how_to/#text-splitters): 如何使用 `DocumentTransformers`
- [Integrations](https://python.langchain.com/v0.2/docs/integrations/document_transformers/)
- [Interface](https://api.python.langchain.com/en/latest/documents/langchain_core.documents.transformers.BaseDocumentTransformer.html): API 调用

## 3. 创建索引: 存储

现在我们需要为我们的 66 个文本块建立索引，以便我们可以在运行时对它们进行搜索。最常见的做法是嵌入每个文档分割的内容，并将这些嵌入插入到向量数据库（或向量存储）中。当我们想要在我们的分割上搜索时，我们获取一个文本搜索查询，并对其进行嵌入，然后执行某种“相似性”搜索，以识别与我们的查询嵌入最相似的存储分割。最简单的相似度度量是余弦相似度 — 我们测量每对嵌入之间的角的余弦（这些嵌入是高维向量）。

我们可以使用[Chroma](https://python.langchain.com/v0.2/docs/integrations/vectorstores/chroma/)向量存储和[OpenAIEmbeddings](https://python.langchain.com/v0.2/docs/integrations/text_embedding/openai/)模型，在一个方法中完成嵌入和存储所有分割文档。

In [ ]:
from langchain_chroma import Chroma
from langchain_openai import OpenAIEmbeddings
vectorstore = Chroma.from_documents(documents=all_splits, embedding=OpenAIEmbeddings())

**API 调用:**[OpenAIEmbeddings](https://api.python.langchain.com/en/latest/embeddings/langchain_openai.embeddings.base.OpenAIEmbeddings.html)

### Go deeper

`Embeddings`: 文本向量嵌入模型的包装器，用于将文本转换为向量。

- [Docs](https://python.langchain.com/v0.2/docs/how_to/embed_text/): 文本嵌入详情
- [Integrations](https://python.langchain.com/v0.2/docs/integrations/text_embedding/): langchain 的向量模型集成，实现了`Embeddings` 接口的对象
- [Interface](https://api.python.langchain.com/en/latest/embeddings/langchain_core.embeddings.Embeddings.html): API 调用

`VectorStore`: 向量数据库的封装，用于存储和查询向量。

- [Docs](https://python.langchain.com/v0.2/docs/how_to/vectorstores/): 如何使用向量数据库
- [Integrations](https://python.langchain.com/v0.2/docs/integrations/vectorstores/): 向量数据库继承，实现了`VectorStore` 接口的对象
- [Interface](https://api.python.langchain.com/en/latest/vectorstores/langchain_core.vectorstores.VectorStore.html): API调用

这完成了链的索引部分。此时，我们拥有一个可查询的向量存储，其中包含博客文章内容的切分部分。针对用户提出的问题，理想情况下我们应该能够返回回答问题的博客文章内容。

## 4. 检索和生成: 检索器

现在让我们编写实际的应用逻辑。我们希望创建一个简单的应用，该应用接收用户提出的问题，搜索与该问题相关的文档，将检索到的文档和初始问题传递给一个模型，并返回一个答案。

首先我们需要定义我们搜索文档的逻辑。LangChain定义了一个Retriever接口，它包装了一个索引，可以根据字符串查询返回相关的文档`Documents` 。

向量存储转换为可执行的检索器，最常见的 `Retriever` 类型是[VectorStoreRetriever](https://python.langchain.com/v0.2/docs/how_to/vectorstore_retriever/)，它利用向量存储的相似度搜索功能来实现检索。任何 `VectorStore` 可以轻松地转换为 `Retriever` 使用`VectorStore.as_retriever()`。

In [ ]:
retriever = vectorstore.as_retriever(search_type="similarity", search_kwargs={"k": 6})
retrieved_docs = retriever.invoke("什么是Tree of Thoughts?")
len(retrieved_docs)

6

In [ ]:
print(retrieved_docs[0].page_content)

Tree of Thoughts (Yao et al. 2023) extends CoT by exploring multiple reasoning possibilities at each step. It first decomposes the problem into multiple thought steps and generates multiple thoughts per step, creating a tree structure. The search process can be BFS (breadth-first search) or DFS (depth-first search) with each state evaluated by a classifier (via a prompt) or majority vote.
Task decomposition can be done (1) by LLM with simple prompting like "Steps for XYZ.\n1.", "What are the subgoals for achieving XYZ?", (2) by using task-specific instructions; e.g. "Write a story outline." for writing a novel, or (3) with human inputs.


### Go deeper

对于检索器（retrieval）向量存储（Vector stores）是最常用的，但是也有其他检索器可以使用。

`Retriever`: 一个传入查询的内容返回相关文本`Document` 列表的类

- [Docs](https://python.langchain.com/v0.2/docs/how_to/#retrievers): 更多关于创建不同检索器的文档:
    - `MultiQueryRetriever` 生成[输入问题的变形](https://python.langchain.com/v0.2/docs/how_to/MultiQueryRetriever/)以提高检索命中率。
    - `MultiVectorRetriever` 生成[嵌入向量的变形](https://python.langchain.com/v0.2/docs/how_to/multi_vector/)，也是为了提高检索命中率。
    - `Max marginal relevance` 选择在检索的文档中相[关性和多样性](https://www.cs.cmu.edu/~jgc/publication/The_Use_MMR_Diversity_Based_LTMIR_1998.pdf)，以避免在重复的上下文中传递。
    - 在矢量存储检索期间，可以使用元数据过滤器对文档进行过滤，例如使用[Self Query retriver](https://python.langchain.com/v0.2/docs/how_to/self_query/)。
- [Integrations](https://python.langchain.com/v0.2/docs/integrations/retrievers/): 与检索服务的集成。
- [Interface](https://api.python.langchain.com/en/latest/retrievers/langchain_core.retrievers.BaseRetriever.html): API 调用。

## 5. 检索和生成: 生成器

让我们把所有内容整合成一个链条，以便接收问题，检索相关文档，构建提示，传递给模型，并解析输出。

我们将使用gpt-3.5-turbo OpenAI聊天模型，但可以替换成任何LangChain LLM或ChatModel。

In [ ]:
%%bash
pip install -qU langchain-openai

In [ ]:
import getpass
import os
os.environ["OPENAI_API_KEY"] = getpass.getpass()
from langchain_openai import ChatOpenAI
llm = ChatOpenAI(model="gpt-3.5-turbo-0125")

我们使用LangChain封装好的[RAG提示词](https://smith.langchain.com/hub/rlm/rag-prompt)模板。

In [ ]:
from langchain import hub
prompt = hub.pull("rlm/rag-prompt")
example_messages = prompt.invoke(
{"context": "filler context", "question": "filler question"}
).to_messages()
example_messages

[HumanMessage(content="You are an assistant for question-answering tasks. Use the following pieces of retrieved context to answer the question. If you don't know the answer, just say that you don't know. Use three sentences maximum and keep the answer concise.\nQuestion: filler question \nContext: filler context \nAnswer:")]

In [ ]:
print(example_messages[0].content)

You are an assistant for question-answering tasks. Use the following pieces of retrieved context to answer the question. If you don't know the answer, just say that you don't know. Use three sentences maximum and keep the answer concise.
Question: filler question 
Context: filler context 
Answer:


我们使用LangChain的 [LCEL](https://python.langchain.com/v0.2/docs/concepts/#langchain-expression-language)  协议定义一个链

这里是实现：

In [ ]:
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough
def format_docs(docs):
  return "\n\n".join(doc.page_content for doc in docs)
rag_chain = (
{"context": retriever | format_docs, "question": RunnablePassthrough()}
| prompt
| llm
| StrOutputParser()
)
for chunk in rag_chain.stream("什么是Tree of Thoughts?"):
  print(chunk, end="", flush=True)

Tree of Thoughts 是一个扩展了 CoT 的框架，由 Yao 等人于 2023 年提出。它通过在每个步骤探索多种推理可能性来拓展问题，首先将问题分解为多个思考步骤，并在每个步骤生成多个思考，从而创建了一棵树状结构。搜索过程可以是 BFS 或 DFS，每个状态都由分类器评估，通过提示或多数投票。

**API 调用:**[StrOutputParser](https://api.python.langchain.com/en/latest/output_parsers/langchain_core.output_parsers.string.StrOutputParser.html) | [RunnablePassthrough](https://api.python.langchain.com/en/latest/runnables/langchain_core.runnables.passthrough.RunnablePassthrough.html)

剖析理解什么是 LCEL

首先：这些组件（`retriever`，`prompt`，`llm`等）都是`Runnable`的实例。这意味着它们实现了相同的方法，比如同步和异步的`.invoke`，`.stream`，或者`.batch`，这使它们更容易连接在一起。它们可以通过`|`操作符连接到[RunnabaleSequence](https://api.python.langchain.com/en/latest/runnables/langchain_core.runnables.base.RunnableSequence.html)（另一个`Runnable`）。

在遇到`|`操作符时，LangChain会自动将某些对象转换为`Runnable`。在这里，`format_docs`被转换为[RunnableLambda](https://api.python.langchain.com/en/latest/runnables/langchain_core.runnables.base.RunnableLambda.html)，包含`context`和`question`的字典被转换为[RunnableParallel](https://api.python.langchain.com/en/latest/runnables/langchain_core.runnables.base.RunnableParallel.html)。只需要记住，那就是每个对象都是`Runnable`。

让我们跟踪上面整个链，怎样输入问题到每一步的执行。

正如我们在上面看到的，提示的输入应该是一个带有 `"context"`和`"question"`键的字典。因此，这个链的第一个元素将从输入的问题中拿到这两个值。

- `retriever | format_docs` 通过检索器传递问题，生成文档对象，然后传递给 `format_docs` 生成字符串；
- `RunnablePassthrough()` 将输入的问题原封不动地传入进来

That is, if you constructed

In [ ]:
chain = (
{"context": retriever | format_docs, "question": RunnablePassthrough()}
| prompt
)

那么，**`chain.invoke(question)`** 将构建一个格式化的提示词，准备进行推理。（注意：在使用 LCEL 开发时，一步一步拼装子链打印结果，非常有用）

链的最后步骤是 **`llm`**，它进行推理，而 **`StrOutputParser()`** 则只是从 LLM 的输出消息中提取字符串内容。

可以通过其 [LangSmith](https://smith.langchain.com/public/1799e8db-8a6d-4eb2-84d5-46e8d7d5a99b/r) 追踪来分析此链的各个步骤。

### 内置链

如果需要，LangChain 包含实现上述 LCEL 的便捷功能。我们组合了两个函数：

- `create_stuff_documents_chain` 指定了如何检索上下文填充到提示词模板和 LLM中。在这种情况下，将把 `context` 内容全部“填充”到提示词中。这就实现了输入键为 `context` （上下文）和 `input` （输入问题）从提示词生成到大模型调用的过程。
- `create_retrieval_chain` 这个链中，相当于在上面定义的链前面加了个检索本地知识库的节点，将检索到的上下文交给上面定义好的链。它的输入键是 `input`，并在其输出中包含了 `input`、`context` 和 `answer`。

In [ ]:
from langchain.chains import create_retrieval_chain
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain_core.prompts import ChatPromptTemplate
system_prompt = (
"You are an assistant for question-answering tasks. "
"Use the following pieces of retrieved context to answer "
"the question. If you don't know the answer, say that you "
"don't know. Use three sentences maximum and keep the "
"answer concise."
"\n\n"
"{context}"
)
prompt = ChatPromptTemplate.from_messages(
[
("system", system_prompt),
("human", "{input}"),
]
)
question_answer_chain = create_stuff_documents_chain(llm, prompt)
rag_chain = create_retrieval_chain(retriever, question_answer_chain)
response = rag_chain.invoke({"input": "What is Task Decomposition?"})
print(response["answer"])


Task decomposition is the process of breaking down a complex task into smaller and more manageable subtasks or steps. This approach enables agents or models to tackle intricate problems by addressing each component separately, facilitating clearer understanding and efficient problem-solving. Techniques like Chain of Thought (CoT) and Tree of Thoughts extend this concept by guiding models to think step by step or explore multiple reasoning possibilities at each step, respectively, enhancing their performance on complex tasks.Task decomposition is the process of breaking down a complex task into smaller and more manageable subtasks or steps. This approach enables agents or models to tackle intricate problems by addressing each component separately, facilitating clearer understanding and efficient problem-solving. Techniques like Chain of Thought (CoT) and Tree of Thoughts extend this concept by guiding models to think step by step or explore multiple reasoning possibilities at each step,

**API 调用:**[create_retrieval_chain](https://api.python.langchain.com/en/latest/chains/langchain.chains.retrieval.create_retrieval_chain.html) | [create_stuff_documents_chain](https://api.python.langchain.com/en/latest/chains/langchain.chains.combine_documents.stuff.create_stuff_documents_chain.html) | [ChatPromptTemplate](https://api.python.langchain.com/en/latest/prompts/langchain_core.prompts.chat.ChatPromptTemplate.html)

这种方法比上面 LCEL 链的优点是能够同时在**输出**中拿到`context` 和 `answer` ，而在 LCEL 链中我们只能拿到相关的输出，需要单独记录相应的上下文。

### 返回来源

在问答应用中，经常需要向用户展示用于生成答案的信息来源。LangChain 内置的 **`create_retrieval_chain`** 将检索到的源文档通过 `context` 键传递到输出中：

In [ ]:
for document in response["context"]:
  print(document)
print()

page_content='Fig. 1. Overview of a LLM-powered autonomous agent system.\nComponent One: Planning#\nA complicated task usually involves many steps. An agent needs to know what they are and plan ahead.\nTask Decomposition#\nChain of thought (CoT; Wei et al. 2022) has become a standard prompting technique for enhancing model performance on complex tasks. The model is instructed to “think step by step” to utilize more test-time computation to decompose hard tasks into smaller and simpler steps. CoT transforms big tasks into multiple manageable tasks and shed lights into an interpretation of the model’s thinking process.' metadata={'source': 'https://lilianweng.github.io/posts/2023-06-23-agent/', 'start_index': 1585}
page_content='Tree of Thoughts (Yao et al. 2023) extends CoT by exploring multiple reasoning possibilities at each step. It first decomposes the problem into multiple thought steps and generates multiple thoughts per step, creating a tree structure. The search process can be B

### Go deeper

### 模型选择

`ChatModel`: 一个 LLM 支持的聊天模型。接收一系列消息并返回一条消息。

- [Docs](https://python.langchain.com/v0.2/docs/how_to/#chat-models)
- [Integrations](https://python.langchain.com/v0.2/docs/integrations/chat/): 25+ 可供选择的集成。
- [Interface](https://api.python.langchain.com/en/latest/language_models/langchain_core.language_models.chat_models.BaseChatModel.html): API 调用。

`LLM`: 一个 text-in-text-out LLM。接受一个字符串并返回一个字符串。

- [Docs](https://python.langchain.com/v0.2/docs/how_to/#llms)
- [Integrations](https://python.langchain.com/v0.2/docs/integrations/llms/): 75+  可供选择的集成。
- [Interface](https://api.python.langchain.com/en/latest/language_models/langchain_core.language_models.llms.BaseLLM.html): API 调用。

### 自定义模板

如上所示，我们可以从直接加载预先设定的提示词(例如，这个 [RAG 提示](https://smith.langchain.com/hub/rlm/rag-prompt)词)。自定义提示词也很容易：

In [ ]:
from langchain_core.prompts import PromptTemplate
template = """Use the following pieces of context to answer the question at the end.
If you don't know the answer, just say that you don't know, don't try to make up an answer.
Use three sentences maximum and keep the answer as concise as possible.
Always say "thanks for asking!" at the end of the answer.
{context}
Question: {question}
Helpful Answer:"""
custom_rag_prompt = PromptTemplate.from_template(template)
rag_chain = (
{"context": retriever | format_docs, "question": RunnablePassthrough()}
| custom_rag_prompt
| llm
| StrOutputParser()
)
rag_chain.invoke("What is Task Decomposition?")

**API 调用:**[PromptTemplate](https://api.python.langchain.com/en/latest/prompts/langchain_core.prompts.prompt.PromptTemplate.html)

[LangSmith trace](https://smith.langchain.com/public/da23c4d8-3b33-47fd-84df-a3a582eedf84/r)

## 总结

我们已经介绍了构建基本问答应用程序的所有步骤：

- 使用[文档加载器](https://python.langchain.com/v0.2/docs/concepts/#document-loaders)加载数据
- 使用[文本拆分器](https://python.langchain.com/v0.2/docs/concepts/#text-splitters)对索引数据进行分块，以便模型更容易使用
- 对数据进行[嵌入](https://python.langchain.com/v0.2/docs/concepts/#embedding-models)，并将数据存储在[向量存储器](https://python.langchain.com/v0.2/docs/how_to/vectorstores/)中
- 根据传入的问题[检索](https://python.langchain.com/v0.2/docs/concepts/#retrievers)先前存储的分块
- 使用检索到的分块作为上下文生成答案